In [1]:
data_var = '2023-12-20'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9382,2023-12-20,Brasil Nbb,19:00,Minas,Brasília,1.04,9.85,162.5,1.87,1.79,-21.5,2.00,1.72,Kj4evQGd,0.961538,0.101523,0.534759,0.558659,0.063061,0.6,0.2,NaN,NaN,108.708,23.355310,0.214844,360.076,240.106060,0.666821,108.474,425.718,70.35,409.53,0.0,0.0,0.0,0.0,1.144097,0.030912,0.106446,-1.20,-0.240,-0.166667,0.000000,0.0,0.000000,-5.00,-1.000,-8.850000,0.000000,0.0,0.000000
9383,2023-12-20,Brasil Nbb,19:30,São José,Bauru,1.45,2.62,155.5,1.85,1.85,-5.5,1.95,1.76,OOUymIKd,0.689655,0.381679,0.540541,0.540541,0.071335,0.6,0.8,NaN,NaN,119.684,18.160395,0.151736,110.024,14.103853,0.128189,187.518,178.672,98.28,126.72,0.0,0.0,0.0,0.0,0.406543,0.000000,0.072426,2.96,0.592,0.760135,0.000000,0.0,0.000000,-3.56,-0.712,-2.275281,0.000000,0.0,0.000000
9384,2023-12-20,Brasil Nbb,20:00,Botafogo,Unifacisa,3.20,1.32,158.5,1.85,1.83,5.5,1.94,1.77,2FTunxz3,0.312500,0.757576,0.540541,0.546448,0.070076,0.6,0.6,NaN,NaN,351.016,123.317149,0.351315,150.148,85.908939,0.572162,253.406,170.706,188.10,129.22,0.0,0.0,0.0,0.0,0.588213,0.007686,0.064802,2.35,0.470,4.680851,0.000000,0.0,0.000000,-3.50,-0.700,-0.457143,0.000000,0.0,0.000000
9385,2023-12-20,Eua Nba,00:00,Golden State Warriors,Boston Celtics,2.98,1.47,233.5,1.86,2.07,5.5,2.01,1.93,ADnVcafq,0.335570,0.680272,0.537634,0.483092,0.015843,0.8,0.4,NaN,NaN,245.710,60.865397,0.247712,164.140,20.598427,0.125493,195.800,164.348,203.36,145.92,0.0,0.0,0.0,0.0,0.479879,0.075569,0.028715,0.96,0.192,10.312500,0.622547,0.5,-0.122547,-0.74,-0.148,-3.175676,0.629264,0.7,0.070736
9386,2023-12-20,Eua Nba,00:00,Portland Trail Blazers,Phoenix Suns,3.44,1.38,229.5,1.80,2.08,6.5,2.06,1.87,OGjZdJ9k,0.290698,0.724638,0.555556,0.480769,0.015335,0.8,0.6,NaN,NaN,304.752,104.515581,0.342953,187.958,41.607089,0.221364,305.866,210.342,282.72,196.35,0.0,0.0,0.0,0.0,0.604415,0.102057,0.068372,-2.75,-0.550,-4.436364,0.376324,0.2,-0.176324,1.93,0.386,0.984456,0.518279,0.6,0.081721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9572,2023-12-20,Europa Wbbl (Fem.),12:00,Aistes LSU Kaunas F,Kibirkstis Vilnius F,9.85,1.04,137.5,1.85,1.85,23.5,2.00,1.72,AJ3VSc7P,0.101523,0.961538,0.540541,0.540541,0.063061,0.0,0.0,NaN,NaN,NaN,NaN,NaN,612.728,786.458868,1.283537,0.000,0.000,572.00,1976.00,1.0,1.0,0.0,0.0,1.144097,0.000000,0.106446,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9573,2023-12-20,Itália Série A1 Feminina,16:30,Oxygen Roma F,San Giovanni F,2.53,1.48,135.5,1.85,1.85,3.5,2.00,1.72,ITWoPrye,0.395257,0.675676,0.540541,0.540541,0.070933,0.0,0.0,NaN,NaN,226.066,221.380817,0.979275,NaN,NaN,NaN,0.000,0.000,108.57,252.59,0.0,0.0,0.0,0.0,0.370305,0.000000,0.106446,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9574,2023-12-20,Rússia Superliga,10:30,Zenit Petersburg 2,Temp Sumz Revda,3.98,1.22,149.5,1.81,1.87,7.5,2.08,1.63,r7pUzLH9,0.251256,0.819672,0.552486,0.534759,0.070928,0.0,0.0,NaN,NaN,NaN,NaN,NaN,91.708,4.800096,0.052341,0.000,0.000,119.18,84.42,0.0,0.0,0.0,0.0,0.750621,0.023058,0.171535,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9575,2023-12-20,Rússia Superliga,13:00,CSKA Moscow 2,Chelyabinsk,1.69,2.09,146.5,1.82,1.88,-4.5,1.94,1.75,dAlYZvXF,0.591716,0.478469,0.549451,0.531915,0.070185,0.6,0.0,NaN,NaN,154.580,4

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
3,00:00,Eua Nba,Golden State Warriors,Boston Celtics,233.5,1.86,1.0000,Over
4,00:00,Eua Nba,Portland Trail Blazers,Phoenix Suns,229.5,1.80,0.9999,Over
6,21:00,Eua Nba,Indiana Pacers,Charlotte Hornets,247.5,1.97,0.5523,Over
14,15:00,Europa Euroliga,Monaco,Panathinaikos,160.5,1.85,1.0000,Over
77,23:00,Eua Ncaa,Oklahoma,North Carolina,154.5,1.91,0.8404,Over
